<a href="https://colab.research.google.com/github/tgalkovskyi/deep-text-recognition-benchmark/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This can be executed in https://colab.research.google.com "Python 3 / GPU" runtime.

In [3]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark
%cd deep-text-recognition-benchmark

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 376 (delta 20), reused 13 (delta 5), pack-reused 336
Receiving objects: 100% (376/376), 2.43 MiB | 2.61 MiB/s, done.
Resolving deltas: 100% (222/222), done.
/content/deep-text-recognition-benchmark


Next, download large model files from Google Drive, using hack: https://stackoverflow.com/questions/20665881/direct-download-from-google-drive-using-google-drive-api/32742700#32742700

In [1]:
models = {
    'None-ResNet-None-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
    'None-VGG-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1GGC2IRYEMQviZhqQpbtpeTgHO_IXWetG',
    'None-VGG-None-CTC.pth': 'https://drive.google.com/open?id=1FS3aZevvLiGF1PFBm5SkwvVcgI6hJWL9',
    'TPS-ResNet-BiLSTM-Attn-case-sensitive.pth': 'https://drive.google.com/open?id=1ajONZOgiG9pEYsQ-eBmgkVbMDuHgPCaY',
    'TPS-ResNet-BiLSTM-Attn.pth': 'https://drive.google.com/open?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9',
    'TPS-ResNet-BiLSTM-CTC.pth': 'https://drive.google.com/open?id=1FocnxQzFBIjDT2F9BkNUiLdo1cC3eaO0',
}

for k, v in models.items():
  doc_id = v[v.find('=')+1:]
  !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=$doc_id" > /tmp/intermezzo.html
  !curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > $k

!ls -al *.pth

# !bash download_models.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3275    0  3275    0     0   4033      0 --:--:-- --:--:-- --:--:--  4033
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    761      0 --:--:-- --:--:-- --:--:--   761
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  186M    0  186M    0     0  5482k      0 --:--:--  0:00:34 --:--:--  9.8M  0  5115k      0 --:--:--  0:00:18 --:--:-- 5753k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    171      0 --:--:--  0:00:02 --:--:--   171
  

In [3]:
output = !CUDA_VISIBLE_DEVICES=0 python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

print(output)

['model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn', 'loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth', '--------------------------------------------------------------------------------', 'image_path               \tpredicted_labels         \tconfidence score', '--------------------------------------------------------------------------------', 'demo_image/demo_1.png    \tavailable                \t0.9999', 'demo_image/demo_2.jpg    \tshakeshack               \t0.9533', 'demo_image/demo_3.png    \tlondon                   \t0.9840', 'demo_image/demo_4.png    \tgreenstead               \t0.9985', 'demo_image/demo_5.png    \ttoast                    \t0.9961', 'demo_image/demo_6.png    \tmerry                    \t0.9975', 'demo_image/demo_7.png    \tunderground              \t1.0000', 'demo_image/demo_8.jpg    \tronaldo                  \t0.8387', 'demo_image/demo_9.jpg    \tbally                    \t0.7566', 'demo_image/demo_10.jpg   \tuniversity        

In [7]:
from IPython.core.display import display, HTML
from PIL import Image
import base64
import io
import pandas as pd

data = pd.DataFrame()
for ind, row in enumerate(output[output.index('image_path               \tpredicted_labels         \tconfidence score')+2:]):
  row = row.split('\t')
  filename = row[0].strip()
  label = row[1].strip()
  conf = row[2].strip()
  img = Image.open(filename)
  img_buffer = io.BytesIO()
  img.save(img_buffer, format="PNG")
  imgStr = base64.b64encode(img_buffer.getvalue()).decode("utf-8") 

  data.loc[ind, 'img'] = '<img src="data:image/png;base64,{0:s}">'.format(imgStr)
  data.loc[ind, 'id'] = filename
  data.loc[ind, 'label'] = label
  data.loc[ind, 'conf'] = conf

html_all = data.to_html(escape=False)
display(HTML(html_all))

,img,id,label,conf
0,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_1.png,available,0.9999
1,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_2.jpg,shakeshack,0.9533
2,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_3.png,london,0.9840
3,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_4.png,greenstead,0.9985
4,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_5.png,toast,0.9961
5,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_6.png,merry,0.9975
6,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_7.png,underground,1.0000
7,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_8.jpg,ronaldo,0.8387
8,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_9.jpg,bally,0.7566
9,"<img src=""data:image/png;base64,iVBORw0KGgoAAA...",demo_image/demo_10.jpg,university,0.9998
